In [5]:
import torch

from PIL import Image
from datasets import load_dataset
from transformers import AutoImageProcessor, AutoModelForImageClassification

/home/jsh/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model = {}

In [7]:
# dataset = load_dataset("huggingface/cats-image")
# image = dataset["test"]["image"][0]
image = Image.open("images/tiger_cat.png")

In [8]:
model_path = "/home/jsh/volume/pretrained-models/resnet-50"
model["image_processor"] = AutoImageProcessor.from_pretrained(model_path)
model["model"] = AutoModelForImageClassification.from_pretrained(
        model_path,
        device_map="cuda:7",
        torch_dtype=torch.float16,
        trust_remote_code=True,
    )

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [19]:
inputs = model["image_processor"](image, return_tensors="pt").to("cuda:7")
inputs = {key: value.half() for key, value in inputs.items()}
outputs = model["model"](**inputs)
predicted_label = outputs.logits.argmax(-1).item()
model["model"].config.id2label[predicted_label]

/home/jsh/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


'tiger cat'

In [20]:
import requests
from glob import glob

In [21]:
for img in glob("images/*"):
    with open(img, "rb") as f:
        res = requests.post(f"http://127.0.0.1:1047/infer", files={"file": f})
        print(res.json())

{'label': 'killer whale, killer, orca, grampus, sea wolf, Orcinus orca', 'elapsed_time': 0.7340381145477295, 'used_memory': 0.22265625}
{'label': 'king crab, Alaska crab, Alaskan king crab, Alaska king crab, Paralithodes camtschatica', 'elapsed_time': 0.02501678466796875, 'used_memory': 0.34765625}
{'label': 'albatross, mollymawk', 'elapsed_time': 0.02510833740234375, 'used_memory': 0.34765625}
{'label': 'king penguin, Aptenodytes patagonica', 'elapsed_time': 0.028003692626953125, 'used_memory': 0.34765625}
{'label': 'tiger cat', 'elapsed_time': 0.04780697822570801, 'used_memory': 0.34765625}
